# Momentum Picks Analyzer - Notebook interactif

Ce notebook vous permet d'explorer et d'utiliser le screener d'actions basé sur les facteurs **Momentum** et **Quality** de manière interactive. Vous pourrez:

- Charger des données pour différents indices boursiers
- Calculer les scores de Momentum et Quality
- Filtrer et trier les actions selon ces critères
- Visualiser les résultats à l'aide de graphiques interactifs

## Configuration initiale

In [ ]:
# Import des modules nécessaires
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

# Import des modules du projet
from data_loader import DataLoader
from momentum import MomentumCalculator
from quality import QualityCalculator
from screener import StockScreener
from visualization import ResultVisualizer
from config import AVAILABLE_INDICES, MOMENTUM_PERIODS, QUALITY_METRICS

# Configuration pour les graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_theme(style="darkgrid")

## Configuration de l'API Alpha Vantage

Pour utiliser ce notebook, vous devez disposer d'une clé API Alpha Vantage. Vous pouvez en obtenir une gratuitement sur [leur site](https://www.alphavantage.co/support/#api-key).

In [ ]:
# Définir votre clé API Alpha Vantage
# Vous pouvez soit l'entrer directement ici, soit la définir comme variable d'environnement
ALPHA_VANTAGE_API_KEY = os.environ.get('ALPHA_VANTAGE_API_KEY', '')

if not ALPHA_VANTAGE_API_KEY:
    ALPHA_VANTAGE_API_KEY = input("Entrez votre clé API Alpha Vantage: ")

# Initialiser le chargeur de données
loader = DataLoader(api_key=ALPHA_VANTAGE_API_KEY)

## 1. Sélection et chargement des données

Choisissez l'indice boursier que vous souhaitez analyser. Les options disponibles sont:

In [ ]:
# Afficher les indices disponibles
print("Indices disponibles:")
for code, name in AVAILABLE_INDICES.items():
    print(f"- {code}: {name}")

# Sélectionner l'indice
selected_index = "SP500"  # Par défaut, vous pouvez changer cette valeur

In [ ]:
# Charger les données pour l'indice sélectionné
print(f"Chargement des données pour l'indice {selected_index}...")
stock_data = loader.load_data_for_index(selected_index, with_fundamentals=True)

if not stock_data:
    print("Aucune donnée n'a pu être chargée. Vérifiez votre connexion et votre clé API.")
else:
    print(f"Données chargées pour {len(stock_data)} actions.")
    
    # Afficher un échantillon de données
    sample_symbol = list(stock_data.keys())[0]
    print(f"\nExemple de données pour {sample_symbol}:")
    
    # Afficher les prix historiques
    if 'historical_prices' in stock_data[sample_symbol]:
        historical_df = stock_data[sample_symbol]['historical_prices']
        display(historical_df.head())
    
    # Afficher les données fondamentales
    if 'fundamentals' in stock_data[sample_symbol]:
        print("\nDonnées fondamentales:")
        display(pd.Series(stock_data[sample_symbol]['fundamentals']))

## 2. Calcul des scores de Momentum et Quality

Calculons maintenant les scores de Momentum et Quality pour chaque action.

In [ ]:
# Initialiser les calculateurs
momentum_calc = MomentumCalculator()
quality_calc = QualityCalculator()

# Calculer les scores pour chaque action
print("Calcul des scores Momentum et Quality...")
results = []

for symbol, data in stock_data.items():
    try:
        # Calcul du score Momentum
        momentum_scores = momentum_calc.calculate_momentum_score(data)
        stock_data[symbol]['momentum'] = momentum_scores
        
        # Calcul du score Quality
        quality_scores = quality_calc.calculate_quality_score(data)
        stock_data[symbol]['quality'] = quality_scores
        
        # Stocker les résultats
        results.append({
            'Symbol': symbol,
            'Name': data.get('name', ''),
            'Sector': data.get('sector', ''),
            'Momentum_Score': momentum_scores.get('total_score', 0),
            'Quality_Score': quality_scores.get('total_score', 0)
        })
    except Exception as e:
        print(f"Erreur lors du calcul des scores pour {symbol}: {str(e)}")

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Afficher un aperçu des résultats
if not results_df.empty:
    print(f"Scores calculés pour {len(results_df)} actions.")
    display(results_df.head())
else:
    print("Aucun résultat n'a pu être calculé.")

## 3. Screening et sélection des meilleures actions

Utilisons maintenant le screener pour combiner les scores, filtrer les actions et sélectionner les meilleures.

In [ ]:
# Paramètres de screening
momentum_weight = 0.6
quality_weight = 0.4
min_momentum = 0.5
min_quality = 0.5
min_combined = 0.6
top_n = 20

# Initialiser le screener
screener = StockScreener(
    momentum_weight=momentum_weight,
    quality_weight=quality_weight
)

# Filtrer les actions sans scores
filtered_data = {symbol: data for symbol, data in stock_data.items() 
                if data.get('momentum') is not None and data.get('quality') is not None}

print(f"Exécution du screening sur {len(filtered_data)} actions...")

# Exécuter le screening
final_results_df, sorted_stocks = screener.screen_stocks(
    filtered_data,
    min_momentum=min_momentum,
    min_quality=min_quality,
    min_combined=min_combined,
    top_n=top_n
)

# Afficher les résultats
if not final_results_df.empty:
    print(f"\nTop {top_n} actions sélectionnées:")
    display(final_results_df[['Symbol', 'Name', 'Sector', 'Combined_Score', 'Momentum_Score', 'Quality_Score']].head(top_n))
else:
    print("Aucune action n'a passé les critères de filtrage.")

## 4. Visualisation des résultats

Créons des visualisations interactives pour mieux comprendre les résultats.

In [ ]:
# Scatter plot interactif des scores Momentum vs Quality
if not final_results_df.empty:
    fig = px.scatter(
        final_results_df, 
        x="Momentum_Score", 
        y="Quality_Score", 
        color="Combined_Score",
        size="Combined_Score",
        hover_name="Symbol",
        hover_data=["Name", "Sector"],
        title=f"Momentum vs Quality Scores - {AVAILABLE_INDICES[selected_index]}",
        color_continuous_scale="viridis"
    )
    
    # Améliorer la mise en page
    fig.update_layout(
        width=900,
        height=700,
        xaxis_title="Score Momentum",
        yaxis_title="Score Quality",
        coloraxis_colorbar_title="Score combiné"
    )
    
    fig.show()

In [ ]:
# Graphique en barres des meilleurs scores combinés
if not final_results_df.empty:
    top_10 = final_results_df.sort_values('Combined_Score', ascending=False).head(10)
    
    fig = go.Figure()
    
    # Ajouter les barres pour le score de Momentum
    fig.add_trace(go.Bar(
        x=top_10['Symbol'],
        y=top_10['Momentum_Score'],
        name='Score Momentum',
        marker_color='rgb(55, 83, 109)'
    ))
    
    # Ajouter les barres pour le score de Quality
    fig.add_trace(go.Bar(
        x=top_10['Symbol'],
        y=top_10['Quality_Score'],
        name='Score Quality',
        marker_color='rgb(26, 118, 255)'
    ))
    
    # Ajouter les barres pour le score combiné
    fig.add_trace(go.Bar(
        x=top_10['Symbol'],
        y=top_10['Combined_Score'],
        name='Score Combiné',
        marker_color='rgb(56, 166, 165)'
    ))
    
    # Mise en page
    fig.update_layout(
        title=f"Top 10 actions - {AVAILABLE_INDICES[selected_index]}",
        xaxis_title="Symbole",
        yaxis_title="Score",
        barmode='group',
        width=900,
        height=600
    )
    
    fig.show()

In [ ]:
# Analyse par secteur
if not final_results_df.empty and 'Sector' in final_results_df.columns:
    # Calculer les statistiques par secteur
    sector_stats = final_results_df.groupby('Sector')[
        ['Momentum_Score', 'Quality_Score', 'Combined_Score']
    ].agg(['mean', 'count', 'std']).reset_index()
    
    # Mettre en forme pour Plotly
    sector_stats_flat = pd.DataFrame({
        'Sector': sector_stats['Sector'],
        'Momentum_Mean': sector_stats['Momentum_Score']['mean'],
        'Quality_Mean': sector_stats['Quality_Score']['mean'],
        'Combined_Mean': sector_stats['Combined_Score']['mean'],
        'Count': sector_stats['Momentum_Score']['count']
    })
    
    # Trier par score combiné moyen
    sector_stats_flat = sector_stats_flat.sort_values('Combined_Mean', ascending=False)
    
    # Créer un graphique en barres par secteur
    fig = px.bar(
        sector_stats_flat,
        x='Sector',
        y=['Momentum_Mean', 'Quality_Mean', 'Combined_Mean'],
        title=f"Scores moyens par secteur - {AVAILABLE_INDICES[selected_index]}",
        barmode='group',
        color_discrete_sequence=['rgb(55, 83, 109)', 'rgb(26, 118, 255)', 'rgb(56, 166, 165)'],
        hover_data=['Count']
    )
    
    # Mise en page
    fig.update_layout(
        xaxis_title="Secteur",
        yaxis_title="Score moyen",
        width=1000,
        height=600,
        legend_title="Métrique"
    )
    
    fig.show()

## 5. Analyse détaillée d'une action spécifique

Examinons en détail l'une des actions sélectionnées.

In [ ]:
# Sélectionner une action à analyser en détail
if not final_results_df.empty:
    # Choisir la meilleure action par défaut
    top_stock_symbol = final_results_df.iloc[0]['Symbol']
    
    # Vous pouvez changer cette valeur pour analyser une autre action
    stock_to_analyze = top_stock_symbol
    
    print(f"Analyse détaillée de {stock_to_analyze}:")
    
    if stock_to_analyze in stock_data:
        stock_info = stock_data[stock_to_analyze]
        
        # Informations générales
        print(f"Nom: {stock_info.get('name', '')}")
        print(f"Secteur: {stock_info.get('sector', '')}")
        
        # Scores calculés
        momentum_info = stock_info.get('momentum', {})
        quality_info = stock_info.get('quality', {})
        
        print("\nScores Momentum:")
        display(pd.Series(momentum_info))
        
        print("\nScores Quality:")
        display(pd.Series(quality_info))
        
        # Visualisation des prix historiques
        if 'historical_prices' in stock_info:
            prices_df = stock_info['historical_prices']
            
            fig = go.Figure()
            
            # Ajouter la courbe des prix
            fig.add_trace(go.Scatter(
                x=prices_df.index,
                y=prices_df['close'],
                mode='lines',
                name='Prix de clôture',
                line=dict(color='rgb(49, 130, 189)', width=2)
            ))
            
            # Ajouter le volume
            fig.add_trace(go.Bar(
                x=prices_df.index,
                y=prices_df['volume'],
                name='Volume',
                marker=dict(color='rgba(58, 71, 80, 0.5)'),
                yaxis='y2'
            ))
            
            # Mise en page
            fig.update_layout(
                title=f"Historique des prix - {stock_to_analyze}",
                xaxis_title="Date",
                yaxis_title="Prix",
                yaxis2=dict(
                    title="Volume",
                    overlaying="y",
                    side="right"
                ),
                width=1000,
                height=600
            )
            
            fig.show()
    else:
        print(f"Action {stock_to_analyze} non trouvée dans les données.")

## 6. Exportation des résultats

Sauvegardons les résultats pour une utilisation ultérieure.

In [ ]:
# Exporter les résultats au format CSV
if not final_results_df.empty:
    from datetime import datetime
    
    # Créer un répertoire pour les résultats
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    
    # Timestamp pour le nom de fichier
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{output_dir}/momentum_picks_{selected_index}_{timestamp}.csv"
    
    # Enregistrer le fichier
    final_results_df.to_csv(filename, index=False)
    print(f"Résultats enregistrés dans {filename}")

## Conclusion

Ce notebook vous a permis d'explorer le screener Momentum Picks Analyzer de manière interactive. Vous pouvez modifier les paramètres de screening, changer d'indice boursier ou analyser d'autres actions en adaptant le code ci-dessus selon vos besoins.

Pour toute question ou suggestion d'amélioration, n'hésitez pas à créer une issue sur le dépôt GitHub.